In [1]:
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time

In [2]:
airport_staids = {
    'JFK': 74486
    ,'MCO': 72205
    ,'TPA': 72211
           }

In [3]:
period_start = "2024-07-01"
period_end = "2024-08-31"

config = dotenv_values()
api_key = config['RAPIDAPI_key']

In [ ]:
weather_dict = {'extracted_at':[], 
                'airport_code':[], 
                'station_id':[], 
                'extracted_data':[]
               }


url = "https://meteostat.p.rapidapi.com/stations/daily"

headers = {                  
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

for airport in airport_staids:
   
    querystring = {
        "station":airport_staids[airport]
        ,"start":period_start
        ,"end":period_end
        ,"model":"true"
    }
    
    response = requests.get(url, headers=headers, params=querystring) 
                
    weather_dict['extracted_at'].append(datetime.now())                
    weather_dict['airport_code'].append(airport)                      
    weather_dict['station_id'].append(airport_staids[airport])        
    weather_dict['extracted_data'].append(json.loads(response.text))  

    time.sleep(0.5)

In [5]:
weather_daily_df = pd.DataFrame(weather_dict)
weather_daily_df

,extracted_at,airport_code,station_id,extracted_data
0,2025-03-21 12:00:55.793570,JFK,74486,"{'meta': {'generated': '2025-03-21 11:00:55'},..."
1,2025-03-21 12:00:56.508869,MCO,72205,"{'meta': {'generated': '2025-03-21 11:00:56'},..."
2,2025-03-21 12:00:57.202812,TPA,72211,"{'meta': {'generated': '2025-03-21 11:00:57'},..."


In [6]:
pd.json_normalize(weather_daily_df['extracted_data'][0], record_path='data')

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2024-07-01 00:00:00,23.3,17.8,27.8,0.0,0.0,352.0,24.8,None,1016.2,None
1,2024-07-02 00:00:00,23.6,18.3,27.8,0.0,0.0,231.0,12.6,None,1023.3,None
2,2024-07-03 00:00:00,22.3,17.8,26.7,0.0,0.0,176.0,16.2,None,1021.9,None
3,2024-07-04 00:00:00,22.9,20.0,27.8,1.8,0.0,192.0,15.8,None,1014.4,None
4,2024-07-05 00:00:00,24.3,21.7,28.3,0.8,0.0,174.0,12.2,None,1009.9,None
...,...,...,...,...,...,...,...,...,...,...,...
57,2024-08-27 00:00:00,23.8,19.4,28.9,0.0,0.0,144.0,12.2,None,1019.1,None
58,2024-08-28 00:00:00,26.6,22.2,35.0,0.0,0.0,233.0,15.1,None,1014.8,None
59,2024-08-29 00:00:00,24.9,21.1,27.8,0.0,0.0,92.0,18.0,None,1022.0,None
60,2024-08-30 00:00:00,21.1,16.7,24.4,7.1,0.0,100.0,16.9,None,1024.8,None


In [ ]:
config = dotenv_values()
 
pg_user = config['POSTGRES_USER'] 
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [8]:
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

engine = create_engine(url, echo=False)

In [9]:
dtype_dict = {
    'extracted_at':types.DateTime,
    'airport_code': types.String,
    'station_id': types.Integer,
    'extracted_data':postgres_json
             }

In [10]:
weather_daily_df.to_sql(name = 'weather_daily_raw', 
                       con = engine, 
                       schema = pg_schema,
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

3